## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Анализ производится с данными в формате CSV впредставленные в виде датафрэйм структуры. В данных находится информация о пользователях услуг кредитования. Данные имеют пропуски, не нормированных значения и не котегорезированная информация.
Предполагается произвести предобработку данных для дальнейшего удобства анализа и сгруппировать данные чтобы ответить на поставленные вопросы.


<a id="id0"></a>
Содержание: 

[1. **Шаг 1. Откройте файл с данными и изучите общую информацию.**](#id1)<br>
[2. **Шаг 2. Предобработка данных.**](#id2)<br>
 [ --обработка пропусков](#id3)<br>
 [ --замена типа данных](#id4)<br>
 [ --обработка дубликатов](#id5)<br>
 [ --лемматизация](#id6)<br>
 [ --категоризация данных](#id7)<br>
[3. **Шаг 3. Ответьте на вопросы**](#id8)<br>
[4. **Шаг 4. Общий вывод**](#id9)<br>
[**Чек-лист готовности проекта**](#id10)<br>

<a id="id1"></a>
### Шаг 1. Откройте файл с данными и изучите общую информацию. 
[к содержанию](#id0)<br>

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from collections import Counter
try:
    #df = pd.read_csv('C:\projects\data.csv')
    df = pd.read_csv(r'/datasets/data.csv') # Код ревьюера
except:
    print('Файл по указанному пути не найден')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод 
1 Имеются пропуски в столбцах: общего трудового стажа(days_employed) и ежемесячного дохода(total_income).

2 В столбце образования(education) имеются данные с разным регистром

3 В столбце целей(purpose) имеются описания не приведенные к конкретным категориям

4 В столбце с возрастом имеются значения равные нулю, таких значений меньше процента относительно всей таблицы, поэтому данные можно откинуть без вреда.

5 В столбце с количеством детей имеются отрицательные значения и значения равные 20

<a id="id2"></a>
### Шаг 2. Предобработка данных
[к содержанию](#id0)<br>

<a id="id3"></a>
### Обработка пропусков
[к содержанию](#id0)<br>

- Строки имеющие отрицательные значения в столбце количества детей будут удалены, тк количество меньше одного процента относительно всех данных. Строки со значениями 20 в столбце с количеством детей также меньше процента относительно всех данных, однако здесь вероятна, пользовательская ошибка заполнения формы, тк на клавиатуре цифры '2' и '0' на соседних кнопках и одной вертикали, поэтому значения будут заменены на 2.
- Строки имеющие нулевые значения в графе возраста будут удалены, тк не представляется возможности возместить за счет других данных или обогатить за счет сторонних источников, также количество этих данных не привышает 1%, поэтому удаление не повлияет на результат анализа.

In [2]:
if df[df['dob_years'] == 0]['dob_years'].count() / df.shape[0] *100 <1: #условие что нулевых значений меньше одного процента во всей табл.
    df.drop(df[df['dob_years'] == 0].index,inplace = True) #удаление строк с нулевыми значениями в столбце возраста 
else: print('Нулевых дначений больше одного процента')    
df.drop(df[df['children'] < 0].index,inplace = True) #удаление строк с отрицательными значениями в столбце количества детей 
df.loc[df['children'] == 20,'children']= 2 #исправление ошибки 20 на 2

count_NaN_days_employed = df['days_employed'].isnull().sum()
count_NaN_total_income = df['total_income'].isnull().sum()
print('Количество пропусков данных типа NaN в столбцах: \n -общего трудового стажа(days_employed), равно',
      count_NaN_days_employed, 
      '\n -ежемесячного дохода(total_income), равно',
      count_NaN_total_income)
if count_NaN_total_income == count_NaN_days_employed:
    print()

Количество пропусков данных типа NaN в столбцах: 
 -общего трудового стажа(days_employed), равно 2161 
 -ежемесячного дохода(total_income), равно 2161



Количество пропущенных данных одинаково, имеется предположение, что пропуски двух столбцов находятся в одних и тех же строках, проверяем:

In [3]:
print(((df['days_employed'] == 'NaN') == (df['total_income'] == 'NaN')).sum())

21377


/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Предположение оказалось верным, NaN  обоих столбцов находятся на одних и тех же строках. Далее рассмотрим таблицу состоящую и строк с пропущенными значениями, что бы выяить возможные закономерности.

In [4]:
df[df['days_employed'].isnull()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Проанализировав выборку строк, с отсутствующими данными, закономенрности относительно данных в других столбцах не выявляны.

In [5]:
print('Процент пропущеных значений ко всей таблице, составляет {:.0%}'.format(count_NaN_days_employed/df.shape[0]))

Процент пропущеных значений ко всей таблице, составляет 10%


Это значительная часть всех данных, поэтому отбросить их, не представляется возможным.
Причинами отсутствия стажа и зароботной платы, у клиентов могут являться сложившиеся жизненные обстоятельства, например, жены военнослужащих не могут найти себе работу по причине ее отсутствия в закрытых гарнизонах, и они вынуждены вести домашнее хозяйство, которое не считается стажем. Также проблемы с трудоустройством существуют в северных и отдаленных регионах.

Однако, выборка людей с пропусками в данных имеет равновесие в уровне образования как высшего, так и среднего, что не исключает предыдущего предположения, но больше склоняет в сторону того, что данные были утеряны на этапе формирования таблицы или сбора данных, подтверждая это наличие пропусков в одних и тех же столбцах. 

Прибегним к небольшому допущению, что данные были собраны в одном густонаселенном регионе, тогда  согласно статье 'Анализ влияния уровня образования на заработную плату работающих: территориальный аспект' Устиновой К. А справедливо следующее, что имеется зависимость между уровнем образования, профессией и уровнем дохода восполнение пропусков будет производится за счет средних значений по категориям профессий и уровню образования, такой путь выбран для повышения точности значений, но тк в столбце days_employed значения не нормированы, временно производится замена на 0. 

In [6]:
df['education'] = df['education'].str.lower() #приведение букв к общему регистру
for i in ['total_income','days_employed']:
    df[i] = df[i].fillna(0)         
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 12 columns):
children            21377 non-null int64
days_employed       21377 non-null float64
dob_years           21377 non-null int64
education           21377 non-null object
education_id        21377 non-null int64
family_status       21377 non-null object
family_status_id    21377 non-null int64
gender              21377 non-null object
income_type         21377 non-null object
debt                21377 non-null int64
total_income        21377 non-null float64
purpose             21377 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB
None


Значения NaN, были успешно заменены, однако  в столбце days_employed имеются отрицательные и экстримально высокие значения. Рассмотрим их.

In [7]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Экстремальные значения,  вероятней всего указаны в часах тк. при разделении на 24 получаются вполне сопостовимые велечины с остальными значениями, взятыми по модулю. Что касается значенний взятых со знаком минус, то предполагаю это сделанно намеренно, чтобы разделить стаж указанный в часах со стажем указанный днях, тк положительных соразмерных значений в таблице не имеется.
Далее стаж приводятся к общей размерности -количеству дней.

In [8]:
df.loc[df['days_employed']>0,'days_employed']/=24 #нивилирование экстремально высоких значений
df.loc[df['days_employed']<0,'days_employed']*=-1 #смена знака
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Возвращаясь к замене пропущенных значений в столбцах days_employed и total_income, восполним средними значениями по профессиям и уровню образования. Средний стаж работы так же оценивается по этим пораметрам, тк при расчете стажа для работника, имеются поправочные коэфициенты для различныйх профессий. В случае недостаточности данных(появление уникальных профессии), пропуски будут восполгнятся средним значением по всем профессиям. 

In [9]:
income_type = df['income_type'].unique() 
education = df['education'].unique()
#сводная таблица по уровню образования и типу профессий со средними значениями по доходу total_income
data_pivot_total_incom = df.pivot_table(index=['income_type'], columns = ['education'], values= 'total_income', aggfunc = 'mean')
for i in income_type:
    for j in education:
        if  data_pivot_total_incom[j][i] > 0: #если имеется значение по такой професссии и уровню образования
            df.loc[(df['education'] == j) & (df['income_type']== i) &(df['total_income'] == 0),'total_income']= data_pivot_total_incom[j][i]
#сводная таблица по уровню образования и типу профессий со средними значениями по стажу работы days_employed   
data_pivot_days_employed = df.pivot_table(index=['income_type'], columns=['education'], values='days_employed' ,aggfunc ='mean') 
for i in income_type:
    for j in education:
        if data_pivot_days_employed[j][i] > 0: #если имеется значение по такой професссии и уровню образования
            df.loc[(df['education'] == j) & (df['income_type']== i) &(df['days_employed'] == 0),'days_employed']= data_pivot_days_employed[j][i]
if (df.loc[df['days_employed'] == 0,'days_employed'].count()==0) and (df.loc[df['total_income'] == 0,'total_income'].count()==0):
    print('Данные упешно перезаписаны')        

Данные упешно перезаписаны


### Вывод

Желательно вернуться к источникам данных, выявить там ошибки формах или терминалах заполнения , для предотвращения появления подобных ошибок в будущем.

<a id="id4"></a>
### Замена типа данных
[к содержанию](#id0)<br>

Столбец days_employed имеет значение количества дней, тк доля дня не играет значительной роли для стажа, то тип данных переводится в целочисленные значения(int).
Столбец total_income имеет значение ежемесячного дохода, тк количество копеек в меньшей степени влияяет на уровень дохода, то тип данных переводится в целочисленные значения(int).

In [10]:
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 12 columns):
children            21377 non-null int64
days_employed       21377 non-null int64
dob_years           21377 non-null int64
education           21377 non-null object
education_id        21377 non-null int64
family_status       21377 non-null object
family_status_id    21377 non-null int64
gender              21377 non-null object
income_type         21377 non-null object
debt                21377 non-null int64
total_income        21377 non-null int64
purpose             21377 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Тип данных был заменен, что уменьшит количество визуального шума для анализирующего таблицу.

<a id="id5"></a>
### Обработка дубликатов
[к содержанию](#id0)<br>

Проверка на дубликаты в таблице с учетом регистра (следует табл. education привести к нижнему регистру)
Для удаления дубликатов используется ручной поиск дубликатов с учетом региста.

In [11]:
print('Длинна табслицы с дубликатами =', df.shape[0])
print('Количество дубликотов в таблице =', df.duplicated().sum())

Длинна табслицы с дубликатами = 21377
Количество дубликотов в таблице = 71


In [12]:
try:
    df = df.drop_duplicates().reset_index()
except:
    print('Дубликаты уже были удалены')
print('Длинна табслицы после удаления дубликатов =', df.shape[0])

Длинна табслицы после удаления дубликатов = 21306


### Вывод

Как видно из значения, длинна таблицы изменилась, после удаления дубликатов.
Основные причины возникновения дубликатов - неправильное соеденение данных  из разных источников, что в принципе подтверждается группой строк с пропусками в разных столбцах, так же возможны ошибки пользователей при занесении информации. 

<a id="id6"></a>
### Лемматизация
[к содержанию](#id0)<br>

Приведение уникальных слов из столбца описания целей к леммам, для выяснения основных категорий запросов на кредит и последующего присвоения их для каждой строки.

In [13]:
m = Mystem()
'''
p = set() #множество слов
unique = df['purpose'].unique() #выбираются уникальные слова из столбца 'purpose'
for i in unique:
    y = m.lemmatize(i) #используя библиотеку pymystem3 слова приводятся к словарной форме и добавляются в множество
    for t in y:
        p.add(t)
print(p)
'''
lemmas =m.lemmatize(' '.join(df['purpose']))
print('Выявленные уникальные слова в столбце целей получения кредита: ')
print(*Counter(lemmas))

Выявленные уникальные слова в столбце целей получения кредита: 
покупка   жилье приобретение автомобиль дополнительный образование сыграть свадьба операция с на проведение для семья недвижимость коммерческий жилой строительство собственный подержать свой со заниматься сделка получение высокий подержанный профильный сдача ремонт 



### Вывод

Из этого множества можно в ручную вычленить существительный и принять их за цели взятия кредита:

свадьба, 
недвижимость/жилье, 
автомобиль, 
операция,
ремонт, 
образование,
строительство

<a id="id7"></a>
### Категоризация данных
[к содержанию](#id0)<br>

Замена описаний, значениями категорий в столбце целей получения кредита('purpose')

In [14]:
category = ['свадьба', 'недвижимость', 'автомобиль', 'жилье', 'операция', 'ремонт', 'образование', 'строительство'] #выявленные категории
#создание словаря, где ключ - это основа слова, для перебора по значениям столбца 'purpose', значение - название выявленной категории
russian_stemmer = SnowballStemmer('russian')  

stem_and_lemm = {russian_stemmer.stem(y):y for y,i in zip(category, range(len(category)))} #жилье = недвижимость   
stem_and_lemm['жил'] = 'недвижимость'
def definition_category(value_data):
    for key, value in stem_and_lemm.items():
        if key in value_data:
            return value    
df['purpose'] = df['purpose'].apply(definition_category)
print(df.head(5))

   index  children  days_employed  dob_years education  education_id  \
0      0         1           8437         42    высшее             0   
1      1         1           4024         36   среднее             1   
2      2         0           5623         33   среднее             1   
3      3         3           4124         32   среднее             1   
4      4         0          14177         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

        purpose  
0  недвижимость  
1    автомобиль  
2  недвижимость  
3   образовани

Категаризация данных по уровню дохода 

минимальный 
низкий
средний
высокий 
максимальный
спекстр значений разбивается на 5 равных частей

### Вывод

При внесении данных удобней было бы создать форму выбора категории, для удобства обработки данных, а в случае появления уникальных целей для кредита, своевременно дополнять новую категорию в форму.

<a id="id8"></a>
### Шаг 3. Ответьте на вопросы
[к содержанию](#id0)<br>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Ниже подсчитывается процент возвражения кредита по количеству детей в семье. 

In [15]:
children = df['children'].unique()
children.sort()
print('{: <15}'.format('КОЛ. ДЕТЕЙ'),'{: >10}'.format('ОБЩЕЕ'),'{: >10}'.format('ДОЛЖН.'),'{: >10}'.format('%'))
for i in children:
    s=df[df['children']==i]['debt'].sum()
    c=df[df['children']==i]['debt'].count()
    print('{: <15}'.format(i),'{: >10.0f}'.format(c),'{: >10.0f}'.format(s),'{: >10.0%}'.format(s / c))

КОЛ. ДЕТЕЙ           ОБЩЕЕ     ДОЛЖН.          %
0                    14022       1058         8%
1                     4792        441         9%
2                     2114        202        10%
3                      328         27         8%
4                       41          4        10%
5                        9          0         0%


### Вывод

Из данных заметно, что не большая зависимость имеется, но незначительная, люди с большим количеством детей имеют больший процент задолжностей по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
'''
family_status = df['family_status'].unique()
print('{: <25}'.format('СЕМЕЙНЫЙ СТАТУС'),'{: >5}'.format('ОБЩЕЕ'),'{: >8}'.format('ЗАДОЛЖ.'),'{: >5}'.format('%'))
for i in family_status:
    s=df[df['family_status']==i]['debt'].sum()
    c=df[df['family_status']==i]['debt'].count()
    print('{: <25}'.format(i),'{: >5.0f}'.format(c),'{: >8.0f}'.format(s),'{: >5.0%}'.format(s / c))
'''  
print(df.groupby('family_status')['debt'].mean()*100)

family_status
Не женат / не замужем    9.788455
в разводе                7.197290
вдовец / вдова           6.526316
гражданский брак         9.357576
женат / замужем          7.552402
Name: debt, dtype: float64


In [24]:
df.pivot_table(index = 'family_status', values = 'debt').style.format({'debt': '{:.2%}'})

,debt
family_status,
Не женат / не замужем,9.79%
в разводе,7.20%
вдовец / вдова,6.53%
гражданский брак,9.36%
женат / замужем,7.55%


### Вывод

Как видно из таблицы, имеется зависимость между симейным положением и возвращением кредита - больший процент людей имеющих задолженности по кредиту приходится на людей не состоящих в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для удобства анализа Уровень дохода будет разбит на категории дохода(интервал доходов на равные 5 частей):

    - минимальный 
    - низкий
    - средний 
    - высокий
    - максимальный
    

In [17]:
df['total_income_category'] = pd.qcut(df['total_income'],5,labels = ['минимальный','низкий','средний','высокий','максимальный']) #интервал дохода разбивается на пять групп 
total_income_category = df['total_income_category'].unique()
print('{: <25}'.format('УРОВЕНЬ ДОХОДА'),'{: >5}'.format('ОБЩЕЕ'),'{: >8}'.format('ЗАДОЛЖ.'),'{: >5}'.format('%'))
for i in total_income_category:
    s=df[df['total_income_category']==i]['debt'].sum()
    c=df[df['total_income_category']==i]['debt'].count()
    print('{: <25}'.format(i),'{: >5.0f}'.format(c),'{: >8.0f}'.format(s),'{: >5.0%}'.format(s / c))    

УРОВЕНЬ ДОХОДА            ОБЩЕЕ  ЗАДОЛЖ.     %
максимальный               4130      293    7%
низкий                     4261      357    8%
средний                    4297      384    9%
минимальный                4262      343    8%
высокий                    4356      355    8%


### Вывод

По данным виндна некоторая незначительная зависимость, что люди с меньшим уровнем дохода чаще иеют задолженность по кредиту. Значение процентного уровня задолженности для людей с высоким и максильным уровнями дахода меньше на 1-2%.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
purpose = df['purpose'].unique()
print('{: <25}'.format('ЦЕЛЬ ПОЛУЧЕНИЯ КРЕДИТА'),'{: >5}'.format('ОБЩЕЕ'),'{: >8}'.format('ЗАДОЛЖ.'),'{: >5}'.format('%'))
for i in purpose:
    s=df[df['purpose']==i]['debt'].sum()
    c=df[df['purpose']==i]['debt'].count()
    print('{: <25}'.format(i),'{: >5.0f}'.format(c),'{: >8.0f}'.format(s),'{: >5.0%}'.format(s / c))

ЦЕЛЬ ПОЛУЧЕНИЯ КРЕДИТА    ОБЩЕЕ  ЗАДОЛЖ.     %
недвижимость              10740      779    7%
автомобиль                 4273      399    9%
образование                3985      370    9%
свадьба                    2308      184    8%


### Вывод

Зависимость между возвращением кредита в срок и целью кредита имеется, однака незначительная 1-2%.

<a id="id9"></a>
### Шаг 4. Общий вывод
[к содержанию](#id0)<br>

Данные изначально были не обработаны - имели пропуски, были отрецательные  и экстримально высокие значения, заполнены нулевыми значениями в графе возраста, а так же тип присутствовали типы данных неудобные для обработки и визуального анализа.
Касательно гипотез:
- Семейное положение имеет кореляцию на вероятность выплаты кредита в срок, люди состоящиие в браке имеют меньший процент задолженности по кредиту
- Между количеством детей в семье и возвращенем кредита в срок имеется не значительная корреляция(1-2%)- люди с большим количеством детей, имеют более высокий процент по задолжености по  кредиту.
- Между уровнем дохода и степенью задолженности людей по кредиту, имеется зависимость - люди с меньшим уровнем дохода имеют более высокий уровень задолженности.
- Цель кредита покупка недвижимости имеет минимальный уровень задолженности, а кредит с целью оплаты образования или покупки автомобиля имеет более высокий процент задолженностей